In [1]:
! pip install -U bitsandbytes accelerate
! pip install git+https://github.com/huggingface/transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 32.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from pydantic import BaseModel, Field
from typing import List, Optional
import torch
import json
from PIL import Image


def resize_document(image_path):
    new_size = (1024, 1024)
    img = Image.open(image_path).convert("RGB")
    resized_image = img.resize(new_size, Image.LANCZOS)
    return resized_image


# Schema definition
class SubjectGrade(BaseModel):
    subject: str
    grade: str


class SecondaryExam(BaseModel):
    exam_type: str
    exam_date: str
    exam_centre_no: str
    index_no: str
    subjects_and_grades: List[SubjectGrade]
    result: str
    aggregate: str


class PSCApplication(BaseModel):
    post_applied_for: str
    ministry_department: str
    date_of_advertisement: str
    national_identity_no: str
    surname: str
    other_names: str
    maiden_name: str
    residential_address: str
    date_of_birth: str
    age: str
    place_of_birth: str
    nationality: str
    phone_office: str
    phone_home: str
    phone_mobile: str
    email: str
    secondary_ordinary_level: SecondaryExam

🚨 `image_seq_length` is part of ImagesKwargs, but not documented. Make sure to add it to the docstring of the function in /usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils_fast.py.


In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,            # <-- this enables 4-bit quantization
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",    # NormalFloat4 quantization (recommended)
    bnb_4bit_compute_dtype="float16"  # good balance between speed & accuracy for T4
)

model_name = "Qwen/Qwen3-VL-8B-Instruct"

model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-8B-Instruct",
    # dtype="auto",
    device_map="auto",
    quantization_config=quantization_config
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-8B-Instruct")

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [12]:
def qwen3_vl_ocr_to_schema(image_path: str, schema_class: BaseModel, max_new_tokens: int = 1024):
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_path, # Can be a file path string or PIL Image object
                },
                {
                    "type": "text",
                    "text": f"""
                    Extract all visible text and structured content from this image or PDF.Preserve the original structure as closely as possible — including headings, paragraphs, lists, tables, and emphasis — and output everything in valid Markdown. Guidelines: Keep text order consistent with the visual layout (top to bottom, left to right). Convert any bold, italic, or underlined text to Markdown equivalents (bold, italic, underline). For tables, use Markdown table syntax. For bullet or numbered lists, preserve indentation and order. If there are images, diagrams, or logos, include a Markdown placeholder like ![image: description]. Do not omit any textual element, even if faint or partially visible. If handwriting or unclear text appears, write [unreadable] or your best guess in brackets. Output only the Markdown — no explanations or comments.
                    """
                },
            ],
        }
    ]

    # Prepare inputs for the model
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )
    inputs = inputs.to(model.device)

    # Generate the output
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True, # Use greedy decoding for more consistent structured output
        pad_token_id=processor.tokenizer.eos_token_id,
        temperature=0.1, # Low temperature for consistency if do_sample=True
        # Use if do_sample=True
        top_k=1,
        top_p=0,
        min_p=0,
    )

    # Decode the generated tokens, trimming the input prompt
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    raw_output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]

    print(f"Raw model output:\n{raw_output_text}\n") # Debug print

    # # Attempt to parse the output as JSON and validate against the schema
    # try:
    #     # Sometimes the model might wrap the JSON in markdown code blocks
    #     import re
    #     json_match = re.search(r'```json\n(.*?)\n```', raw_output_text, re.DOTALL)
    #     if json_match:
    #         json_str = json_match.group(1)
    #     else:
    #         # Assume the whole output is JSON
    #         json_str = raw_output_text.strip()

    #     # Validate and create the Pydantic object
    #     structured_data = schema_class.model_validate_json(json_str)
    #     return structured_data

    # except json.JSONDecodeError as e:
    #     print(f"Error decoding JSON from model output: {e}")
    #     print(f"Attempted to decode: {raw_output_text}")
    #     return None
    # except Exception as e:
    #     print(f"Error validating against schema: {e}")
    #     print(f"Output was: {raw_output_text}")
    #     return None

In [6]:
image_path = "/kaggle/input/psc-forms/20250407_150848.jpg"
image = resize_document(image_path)

output_filename = "/kaggle/working/processed_20250407_150848.jpg"
image.save(output_filename)

print(f"Image saved to: {output_filename}")

Image saved to: /kaggle/working/processed_20250407_150848.jpg


In [7]:
applicant_data = qwen3_vl_ocr_to_schema(image, PSCApplication)
if applicant_data:
    print("Structured applicant data:")
    print(applicant_data.model_dump_json(indent=2))
else:
    print("Failed to extract structured data for applicant.")

Raw model output:
{
  "post_applied_for": "TRAMIEE",
  "ministry_department": "EDUCATION AND TOURISM RESOURCES, TERTIARY",
  "date_of_advertisement": "2",
  "national_identity_no": "6216011919101128",
  "surname": "RAMANE RAMANE",
  "other_names": "MAARIE",
  "maiden_name": "RAMANE BIAWCHA",
  "residential_address": "ROUTE ST MICHEL, GRAND MAURE",
  "date_of_birth": "26/10/1918",
  "age": "23",
  "place_of_birth": "SSEM HESPAT",
  "nationality": "MAURITIAN",
  "phone_office": "2616011918",
  "phone_home": "2616011933",
  "phone_mobile": "2616011933",
  "email": "SSEM HESPAT",
  "secondary_ordinary_level": {
    "exam_type": "Cambridge GCE or London General Certificate of Education (Ordinary Level)",
    "exam_date": "11/11/11",
    "exam_centre_no": "01010101",
    "index_no": "01010101",
    "subjects_and_grades": [
      {
        "subject": "MATHEMATICS",
        "grade": "B"
      },
      {
        "subject": "FRENCH",
        "grade": "C"
      },
      {
        "subject": "HIST

In [11]:
new_size = (1600, 1600)
image_path = "/kaggle/input/psc-forms/20250407_150848.jpg"
img = Image.open(image_path).convert("RGB")
resized_image = img.resize(new_size, Image.LANCZOS)

applicant_data = qwen3_vl_ocr_to_schema(resized_image, PSCApplication)
if applicant_data:
    print("Structured applicant data:")
    print(applicant_data.model_dump_json(indent=2))
else:
    print("Failed to extract structured data for applicant.")

Raw model output:
```markdown
P.S.C. Form 7
FOR OFFICIAL USE

REPUBLIC OF MAURITIUS
PUBLIC SERVICE COMMISSION

Application Form

Read carefully the enclosed Notes & Instructions to candidates before filling this form

1. Post Applied for: **TRAINER** (PRIMARY SCHOOL EDUCATOR)

Ministry/Department: EDUCATION AND HUMAN RESOURCES, TERTIARY
Date of Advertisement/Vacancy Circular: 1
Previous Posts Applied for: 1

2. National Identity No. **B1601980900128**

Title Mr ☐ Mrs ☐ Miss ☐ Ms ☐
Marital Status: Married ☐ Single ☐ Other:
Surname: **B1 JOV**
Other Names: **MARIE RAMENA BIANCA** (in block letters)
Maiden Name (if applicable): (in block letters)

3. Residential Address:
**ROUTE ST MICHEL, GRAND GAUGE**

Phone No: Office: **26611918**
Phone No: Home: **2661133**
Mobile: **ST4F662**
Email address: **RAMENA B1 JOV@GMAIL.COM**

Date of Birth: **21/01/1984**
Place of Birth: **SSRN HOSPITAL**
Nationality: **MAURITIAN**
Certificate No. (if Naturalised): & Date

4. SECONDARY ORDINARY LEVEL
State

In [9]:
new_size = (1920, 1920)
image_path = "/kaggle/input/psc-forms/20250407_150848.jpg"
img = Image.open(image_path).convert("RGB")
resized_image = img.resize(new_size, Image.LANCZOS)

applicant_data = qwen3_vl_ocr_to_schema(resized_image, PSCApplication)
if applicant_data:
    print("Structured applicant data:")
    print(applicant_data.model_dump_json(indent=2))
else:
    print("Failed to extract structured data for applicant.")

Raw model output:
{
  "post_applied_for": "TRAINER PRIMARY SCHOOL EDUCATOR",
  "ministry_department": "Ministry/Department EDUCATION AND HUMAN RESOURCES, TERTIARY",
  "date_of_advertisement": "1",
  "national_identity_no": "5260119180128",
  "surname": "BIAJOU",
  "other_names": "MARIE RAMENA BIANCA",
  "maiden_name": "ROUTE ST MICHEL, GRAND GAUGE",
  "residential_address": "",
  "date_of_birth": "216011918",
  "age": "21",
  "place_of_birth": "SSKN HOSPITAL",
  "nationality": "MAURITIAN",
  "phone_office": "2861133",
  "phone_home": "2861133",
  "phone_mobile": "2861133",
  "email": "ramena.bjou@gmail.com",
  "secondary_ordinary_level": {
    "exam_type": "SECONDARY ORDINARY LEVEL",
    "exam_date": "11111H",
    "exam_centre_no": "MU0160",
    "index_no": "010105",
    "subjects_and_grades": [
      {
        "subject": "MATHEMATICS",
        "grade": "B"
      },
      {
        "subject": "FRENCH",
        "grade": "C"
      },
      {
        "subject": "HINDI",
        "grade": "

In [4]:
def image_processor_2(input_path: str, output_path: str) -> str:
    from PIL import Image, ImageOps
    import cv2
    import numpy as np
 
    print(f"Process: {input_path}")
 
    # Load image with PIL
    pil_img = Image.open(input_path)
    pil_img = ImageOps.exif_transpose(pil_img)  # fix orientation
 
    # Convert to grayscale OpenCV
    image = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2GRAY)
 
    # 1. Upscale to make small text larger
    scale_factor = 2.0  # try 2x or even 3x
    image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
 
    # 2. Apply CLAHE (adaptive histogram equalization) to improve contrast
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    image = clahe.apply(image)
 
    # 3. Optional light denoising (skip heavy Gaussian blur!)
    image = cv2.medianBlur(image, 3)
 
    # 4. Adaptive thresholding to make text crisp
    thresh = cv2.adaptiveThreshold(
        image, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        31, 15
    )
 
    # 5. Sharpen edges a bit
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    sharpened = cv2.filter2D(thresh, -1, kernel)
 
    # Save result
    result = Image.fromarray(sharpened)
    # result.save(prefix_data_dir(output_path, False))
    result.save(output_path)
 
 
    return output_path

In [5]:
input_img = "/kaggle/input/psc-forms/20250407_150848.jpg"
output_img = "/kaggle/working/processed_150848.jpg"
image_processor_2(input_img, output_img)

Process: /kaggle/input/psc-forms/20250407_150848.jpg


'/kaggle/working/processed_150848.jpg'

In [11]:
new_size = (1920, 1920)
img = Image.open("/kaggle/working/processed_150848.jpg")
resized_image = img.resize(new_size, Image.LANCZOS)

applicant_data = qwen3_vl_ocr_to_schema(resized_image, PSCApplication)


Raw model output:
```markdown
ICE COMMISSION
P.S.C. Form 7
FOR OFFICIAL USE

Read carefully the enclosed Notes
& Instructions to candidates
before filling this form

# PUBLIC SERVICE COMMISSION
## Application Form

### 1. Post Applied for
**TRAINED PRIMARY SCHOOL EDUCATOR**

Ministry/Department: EDUCATION AND HUMAN RESOURCES, TERTIARY

Date of Advertisement/Vacancy Circular: …………………………………………

Previous Posts Applied for
1. …………………………………………
2. …………………………………………

### 2. National Identity No.
**[BIJ601980900128]**

Title: Mr ☐ Mrs ☐ Miss ☐ Ms ☐

Marital Status: Married ☐ Single ☐ Other: …………………………………………

Surname: **BIJOU** (in block letters)

Other Names: **MARIE RIMENA BIANCA** (in block letters)

Maiden Name (if applicable): …………………………………………

### 3. Residential Address:
(in block letters)
**ROUTE ST MICHEL, GRAND GAUGE**

Phone No: Office ………………………………………… Home 2561133 ………………………………………… Mobile 7476862 Email address: rimena.bijou@gmail.com

Date of Birth: **[26/01/198]** Age **[27]** Place o

In [10]:
torch.cuda.empty_cache()